# Summarize field and remote sensing obs from intensive validation
#### at field transects along ICESat-2/ATLAS transects at Seward sites)

with field observations at ground level (TODO) and from UAV 

https://docs.google.com/spreadsheets/d/1jqP0K0LZmswK4ZZNU6BxKYgjaBZM0HRkUrWS1QFLvIE/edit?gid=0#gid=0  

could be used for manuscript:
https://docs.google.com/document/d/1uj9-xlLTJK1xeGvzIEHvNNnquXYMUNsBDB30a3HhlOc/edit?usp=sharing

#### Workflow:  
+ `code/above-shrubs/notebooks/icesat2_atl03_atl08_class_map_sewardpen.ipynb`
+ `code/above-shrubs/notebooks/field_analysis_build_table.ipynb`
+ `code/above-shrubs/notebooks/field_analysis_plotting_v3.ipynb`
+ `code/above-shrubs/notebooks/field_transect_plots.ipynb`
+ `code/above-shrubs/notebooks/field_summarize.ipynb`

| Author | Affiliation | Date |
| --- | --- | --- |
| Paul Montesano, PhD | Innovation Lab ; NASA Goddard Space Flight Center | Sept. 2024 |

In [1]:
import os
import pandas as pd
import geopandas as gpd
import glob

/explore/nobackup/people/pmontesa/.nccstmp/ipykernel_2384922/2992939443.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [4]:
EXPEDITION_ID = 'sewardpen2024'
OUTDIR = f'/explore/nobackup/people/pmontesa/userfs02/projects/ilab/above_shrubs/data/field/{EXPEDITION_ID}'
OUTDIR_PLOTS = '/explore/nobackup/projects/above/misc/ABoVE_Shrubs/plots'

## Read tables: Raster CHM extractions at field points

In [5]:
# Read seward reference data frame
all_ref_m = pd.read_csv(os.path.join(OUTDIR, 'sewardpen2024_all_rs_validation.csv'))
all_ref_m.shape
#all_ref_gdf_extract = gpd.read_file(os.path.join(OUTDIR, 'sewardpen2024_all_rs_validation_unmelted.gpkg'))

/explore/nobackup/people/pmontesa/.nccstmp/ipykernel_2384922/635288899.py:2: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  all_ref_m = pd.read_csv(os.path.join(OUTDIR, 'sewardpen2024_all_rs_validation.csv'))


(1586060, 19)

In [6]:
all_ref_m.columns

Index(['Unnamed: 0', 'height_ref', 'site', 'type_ref', 'class_name', 'color',
       'beam', 'doy', 'x', 'y', 'source', 'value', 'source_cat',
       'source_mod_dev_app_class', 'source_type', 'source_extent',
       'source_res', 'source_temp', 'source_temp_dif_ref'],
      dtype='object')

In [46]:
all_ref_m.type_ref.unique()

array(['ATL03', 'ATL08', 'ATL08 30m', 'field 0.3m', 'field 2m'],
      dtype=object)

In [47]:
all_ref_m.source_type.unique()

array(['CHM (ref.; UAV)', 'NASA|UMd HLS', 'NASA|UMd S1+HLS',
       'Pub: Tolan et al. 2024', 'Pub: Lang et al. 2023',
       'Pub: Potapov et al. 2021', 'VHR CHM v1.0 (UNet)',
       'VHR CHM v2.1 (ViT DINOv2)', 'VHR CHM v2.2.1 (ViT DINOv2-RS)',
       'VHR CHM v2.2.2 (ViT DINOv2-RS)', 'VHR CHM v2.2.3 (ViT DINOv2-RS)',
       'VHR CHM v2.2.4 (ViT DINOv2-RS)', 'VHR CHM v1.5.1 (UNet)',
       'VHR CHM v1.5.2 (UNet)', 'VHR CHM v3.3.2.1 (DINOv3)',
       'VHR CHM v3.3.2.2 (DINOv3)', 'VHR CHM v3.3.2.3 (DINOv3)',
       'VHR CHM v3.3.2.4 (DINOv3)', 'VHR CHM v4.3.2.2 (DINOv3)',
       'VHR CHM v4.3.2.3 (DINOv3)', 'UAV', 'CHM (training; IFSAR)'],
      dtype=object)

In [9]:
all_ref_m[all_ref_m.source_type == 'VHR CHM v4.3.2.2 (DINOv3)'].head()

,Unnamed: 0,height_ref,site,type_ref,class_name,color,beam,doy,x,y,source,value,source_cat,source_mod_dev_app_class,source_type,source_extent,source_res,source_temp,source_temp_dif_ref
1351540,1353820,0.091385,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632302e+06,ht_20110727_103001000D914900chm_4.3.2.2,-9999.0,Deep Learning,boreal|boreal,VHR CHM v4.3.2.2 (DINOv3),Regional,2m,2010-2020,4-14 yr dif
1351541,1353821,0.095764,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632302e+06,ht_20110727_103001000D914900chm_4.3.2.2,-9999.0,Deep Learning,boreal|boreal,VHR CHM v4.3.2.2 (DINOv3),Regional,2m,2010-2020,4-14 yr dif
1351542,1353822,-0.077309,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20110727_103001000D914900chm_4.3.2.2,-9999.0,Deep Learning,boreal|boreal,VHR CHM v4.3.2.2 (DINOv3),Regional,2m,2010-2020,4-14 yr dif
1351543,1353823,-0.090279,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20110727_103001000D914900chm_4.3.2.2,-9999.0,Deep Learning,boreal|boreal,VHR CHM v4.3.2.2 (DINOv3),Regional,2m,2010-2020,4-14 yr dif
1351544,1353824,-0.021507,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20110727_103001000D914900chm_4.3.2.2,-9999.0,Deep Learning,boreal|boreal,VHR CHM v4.3.2.2 (DINOv3),Regional,2m,2010-2020,4-14 yr dif


In [10]:
all_ref_m[all_ref_m.source == 'ht_20200722_10300100AB1FD100chm_2.2.2'].head()

,Unnamed: 0,height_ref,site,type_ref,class_name,color,beam,doy,x,y,source,value,source_cat,source_mod_dev_app_class,source_type,source_extent,source_res,source_temp,source_temp_dif_ref
456100,458380,0.091385,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632302e+06,ht_20200722_10300100AB1FD100chm_2.2.2,0.0,Deep Learning,boreal|boreal,VHR CHM v2.2.2 (ViT DINOv2-RS),Regional,2m,2010-2020,4-14 yr dif
456101,458381,0.095764,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632302e+06,ht_20200722_10300100AB1FD100chm_2.2.2,0.0,Deep Learning,boreal|boreal,VHR CHM v2.2.2 (ViT DINOv2-RS),Regional,2m,2010-2020,4-14 yr dif
456102,458382,-0.077309,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20200722_10300100AB1FD100chm_2.2.2,0.0,Deep Learning,boreal|boreal,VHR CHM v2.2.2 (ViT DINOv2-RS),Regional,2m,2010-2020,4-14 yr dif
456103,458383,-0.090279,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20200722_10300100AB1FD100chm_2.2.2,0.0,Deep Learning,boreal|boreal,VHR CHM v2.2.2 (ViT DINOv2-RS),Regional,2m,2010-2020,4-14 yr dif
456104,458384,-0.021507,council_01,ATL03,Ground,brown,gt1l,230.0,-1.584183e+07,6.632304e+06,ht_20200722_10300100AB1FD100chm_2.2.2,0.0,Deep Learning,boreal|boreal,VHR CHM v2.2.2 (ViT DINOv2-RS),Regional,2m,2010-2020,4-14 yr dif


## Summarize Field & Remote Sensing data frame

In [33]:
def summarize_obs_by_site(all_ref_m, source_type_filter=None, exclude_filter=None, 
                          filter_field_obs=False):
    """
    Summarize reference observations by site.
    
    Parameters:
    -----------
    all_ref_m : DataFrame
        DataFrame with field and remote sensing observations
    source_type_filter : str or list of str, optional
        String pattern(s) to INCLUDE from 'source_type' column (uses str.contains)
    exclude_filter : str or list of str, optional
        String pattern(s) to EXCLUDE from 'source_type' column (uses str.contains)
    filter_field_obs : bool, default False
        If True, also apply filters to field observations.
        If False, keep ALL field observations regardless of filters.
        
    Returns:
    --------
    dict of DataFrames with various summary statistics
    """
    import pandas as pd
    
    # Apply filters
    df = all_ref_m.copy()
    
    # Identify field observations
    is_field = df['type_ref'].str.contains('field', case=False, na=False)
    
    # Start with all data
    mask = pd.Series([True] * len(df), index=df.index)
    
    # Apply both filters if both are specified
    if source_type_filter is not None and exclude_filter is not None:
        if isinstance(source_type_filter, str):
            source_type_filter = [source_type_filter]
        if isinstance(exclude_filter, str):
            exclude_filter = [exclude_filter]
            
        include_mask = pd.Series([False] * len(df), index=df.index)
        for pattern in source_type_filter:
            include_mask |= df['source_type'].str.contains(pattern, case=False, na=False)
        
        exclude_mask = pd.Series([False] * len(df), index=df.index)
        for pattern in exclude_filter:
            exclude_mask |= df['source_type'].str.contains(pattern, case=False, na=False)
        
        # Apply filter to RS data, optionally to field obs
        if filter_field_obs:
            mask = include_mask & ~exclude_mask
        else:
            mask = is_field | (include_mask & ~exclude_mask)
    
    # Apply only include filter
    elif source_type_filter is not None:
        if isinstance(source_type_filter, str):
            source_type_filter = [source_type_filter]
        
        include_mask = pd.Series([False] * len(df), index=df.index)
        for pattern in source_type_filter:
            include_mask |= df['source_type'].str.contains(pattern, case=False, na=False)
        
        # Apply filter to RS data, optionally to field obs
        if filter_field_obs:
            mask = include_mask
        else:
            mask = is_field | include_mask
    
    # Apply only exclude filter
    elif exclude_filter is not None:
        if isinstance(exclude_filter, str):
            exclude_filter = [exclude_filter]
        
        exclude_mask = pd.Series([False] * len(df), index=df.index)
        for pattern in exclude_filter:
            exclude_mask |= df['source_type'].str.contains(pattern, case=False, na=False)
        
        # Apply filter to RS data, optionally to field obs
        if filter_field_obs:
            mask = ~exclude_mask
        else:
            mask = is_field | (~exclude_mask & ~is_field)
    
    df = df[mask].copy()
    
    # Separate field and RS data for cleaner summaries
    df_field = df[df['type_ref'].str.contains('field', case=False, na=False)]
    df_rs = df[~df['type_ref'].str.contains('field', case=False, na=False)]
    
    summaries = {}
    summaries['filtered_data'] = df
    summaries['filtered_data_field'] = df_field
    summaries['filtered_data_rs'] = df_rs
    
    # 1. Basic counts by site and type_ref
    summary_site_type = df.groupby(['site', 'type_ref']).size().reset_index(name='count')
    summary_site_type_pivot = summary_site_type.pivot(index='site', columns='type_ref', values='count').fillna(0).astype(int)
    summaries['by_site_and_type'] = summary_site_type_pivot
    
    # 2. Counts by site and source_type (RS ONLY)
    if len(df_rs) > 0:
        summary_site_source_type = df_rs.groupby(['site', 'source_type']).size().reset_index(name='count')
        summary_site_source_type_pivot = summary_site_source_type.pivot(index='site', columns='source_type', values='count').fillna(0).astype(int)
        summaries['by_site_and_source_type'] = summary_site_source_type_pivot
    else:
        summaries['by_site_and_source_type'] = pd.DataFrame()
    
    # 3. Counts by site and source (RS ONLY - full source names)
    if len(df_rs) > 0:
        summary_site_source = df_rs.groupby(['site', 'source']).size().reset_index(name='count')
        summary_site_source_pivot = summary_site_source.pivot(index='site', columns='source', values='count').fillna(0).astype(int)
        summaries['by_site_and_source'] = summary_site_source_pivot
    else:
        summaries['by_site_and_source'] = pd.DataFrame()
    
    # 4. Combined source and source_type view (RS ONLY)
    if len(df_rs) > 0:
        summary_source_both = df_rs.groupby(['site', 'source_type', 'source']).size().reset_index(name='count')
        summaries['by_site_source_and_source_type'] = summary_source_both
    else:
        summaries['by_site_source_and_source_type'] = pd.DataFrame()
    
    # 5. Cross-tabulation: type_ref vs source_type for each site (RS ONLY)
    for site in df['site'].unique():
        site_data_rs = df_rs[df_rs['site'] == site]
        if len(site_data_rs) > 0:
            crosstab = pd.crosstab(site_data_rs['type_ref'], site_data_rs['source_type'], margins=True)
            summaries[f'crosstab_source_type_{site}'] = crosstab
            
            # Also create crosstab with source names
            crosstab_source = pd.crosstab(site_data_rs['type_ref'], site_data_rs['source'], margins=True)
            summaries[f'crosstab_source_{site}'] = crosstab_source
    
    # 6. Field obs summary
    if len(df_field) > 0:
        field_summary = df_field.groupby(['site', 'type_ref']).agg({
            'height_ref': ['count', 'mean', 'std', 'min', 'max']
        }).round(2)
        summaries['field_obs_summary'] = field_summary
        
        # Field obs by source_type
        field_source_summary = df_field.groupby(['site', 'source_type']).size().reset_index(name='count')
        summaries['field_obs_by_source_type'] = field_source_summary
    
    # 7. Missing data patterns - source_type (RS ONLY)
    if len(df_rs) > 0 and 'by_site_and_source_type' in summaries and not summaries['by_site_and_source_type'].empty:
        missing_source_types = summaries['by_site_and_source_type'] == 0
        missing_summary_type = missing_source_types.sum(axis=0).sort_values(ascending=False)
        summaries['missing_source_types_count'] = missing_summary_type[missing_summary_type > 0]
    
    # 8. Observation count distribution
    obs_per_site = df.groupby('site').size().reset_index(name='total_obs')
    obs_per_site_rs = df_rs.groupby('site').size().reset_index(name='rs_obs')
    obs_per_site_field = df_field.groupby('site').size().reset_index(name='field_obs')
    
    obs_per_site = obs_per_site.merge(obs_per_site_rs, on='site', how='left')
    obs_per_site = obs_per_site.merge(obs_per_site_field, on='site', how='left')
    obs_per_site = obs_per_site.fillna(0).astype({'rs_obs': int, 'field_obs': int})
    summaries['obs_per_site'] = obs_per_site
    
    # 9. Spatial coverage by source_type (RS ONLY)
    if len(df_rs) > 0:
        spatial_summary_type = df_rs.groupby(['site', 'source_type']).agg({
            'x': ['count', 'nunique'],
            'y': ['count', 'nunique']
        })
        summaries['spatial_summary_by_source_type'] = spatial_summary_type
    
    # 10. Filter summary info
    summaries['filter_info'] = {
        'original_shape': all_ref_m.shape,
        'filtered_shape': df.shape,
        'field_obs': len(df_field),
        'rs_obs': len(df_rs),
        'rows_removed': len(all_ref_m) - len(df),
        'filter_field_obs': filter_field_obs,
        'source_type_filter': source_type_filter,
        'exclude_filter': exclude_filter,
        'unique_source_types_rs': sorted(df_rs['source_type'].dropna().unique().tolist()) if len(df_rs) > 0 else [],
        'unique_source_types_field': sorted(df_field['source_type'].dropna().unique().tolist()) if len(df_field) > 0 else [],
        'unique_sources_rs': sorted(df_rs['source'].dropna().unique().tolist()) if len(df_rs) > 0 else []
    }
    
    return summaries


In [32]:
# # Check what's actually in source_type column
# print("Unique source_type values:")
# print(all_ref_m['source_type'].unique())
# print("\n")

# # Check if source_type has the values you expect
# print("Sample rows:")
# print(all_ref_m[['source', 'source_type', 'type_ref']].head(20))
# print("\n")

# # Test the filter manually
# test_filter = 'v4.3.2.2'
# is_field = all_ref_m['type_ref'].str.contains('field', case=False, na=False)
# matches_filter = all_ref_m['source_type'].str.contains(test_filter, case=False, na=False)

# print(f"Total rows: {len(all_ref_m)}")
# print(f"Field observations: {is_field.sum()}")
# print(f"Rows matching '{test_filter}': {matches_filter.sum()}")
# print(f"Should keep (field OR matches): {(is_field | matches_filter).sum()}")
# print("\n")

# # Check what source_types match the filter
# print(f"Source_types that match '{test_filter}':")
# print(all_ref_m[matches_filter]['source_type'].unique())
# print("\n")

# # Now run the function and check
# summaries = summarize_obs_by_site(all_ref_m, source_type_filter='v4.3.2.2')
# print("After filtering:")
# print(f"Filtered data shape: {summaries['filtered_data'].shape}")
# print(f"Unique source_types in result:")
# print(summaries['filter_info']['unique_source_types'])

In [58]:
all_ref_m.type_ref.unique()

array(['ATL03', 'ATL08', 'ATL08 30m', 'field 0.3m', 'field 2m'],
      dtype=object)

In [63]:

# Usage:
# Keep all field obs, filter RS to only v4.3.2.2
summaries = summarize_obs_by_site(all_ref_m[all_ref_m.type_ref == 'field 2m'], source_type_filter=['v4.3.2.2'], filter_field_obs=True)

# # Now check - this should only show 'VHR CHM v4.3.2.2 (DINOv3)'
# print("RS source_types after filter:")
# print(summaries['filter_info']['unique_source_types_rs'])

# print("\nField source_types (kept regardless of filter):")
# print(summaries['filter_info']['unique_source_types_field'])

# print("\nRS observations by site and source_type:")
# print(summaries['by_site_and_source_type'])

In [64]:
summaries.keys()

dict_keys(['filtered_data', 'filtered_data_field', 'filtered_data_rs', 'by_site_and_type', 'by_site_and_source_type', 'by_site_and_source', 'by_site_source_and_source_type', 'field_obs_summary', 'field_obs_by_source_type', 'obs_per_site', 'filter_info'])

In [65]:
summaries['filter_info']

{'original_shape': (168720, 19),
 'filtered_shape': (11400, 19),
 'field_obs': 11400,
 'rs_obs': 0,
 'rows_removed': 157320,
 'filter_field_obs': True,
 'source_type_filter': ['v4.3.2.2'],
 'exclude_filter': None,
 'unique_source_types_rs': [],
 'unique_source_types_field': ['VHR CHM v4.3.2.2 (DINOv3)'],
 'unique_sources_rs': []}

In [66]:
summaries['by_site_source_and_source_type']

""


In [67]:
summaries['by_site_and_source_type']

""


In [68]:
summaries['spatial_summary_by_source_type']

KeyError: 'spatial_summary_by_source_type'